In [1]:
!wget https://github.com/alexeygrigorev/datasets/raw/refs/heads/master/jamb_exam_results.csv -O jamb_exam.csv

--2024-11-01 22:40:24--  https://github.com/alexeygrigorev/datasets/raw/refs/heads/master/jamb_exam_results.csv
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/alexeygrigorev/datasets/refs/heads/master/jamb_exam_results.csv [following]
--2024-11-01 22:40:24--  https://raw.githubusercontent.com/alexeygrigorev/datasets/refs/heads/master/jamb_exam_results.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 391501 (382K) [text/plain]
Saving to: ‘jamb_exam.csv’

jamb_exam.csv       100%[===================>] 382.33K  --.-KB/s    in 0.008s  

2024-11-01 22:40:24 (45.2 MB/s) - ‘jamb_exam.csv’ saved [39

In [2]:
import pandas as pd
df = pd.read_csv('jamb_exam.csv')
df.head()

,JAMB_Score,Study_Hours_Per_Week,Attendance_Rate,Teacher_Quality,Distance_To_School,School_Type,School_Location,Extra_Tutorials,Access_To_Learning_Materials,Parent_Involvement,IT_Knowledge,Student_ID,Age,Gender,Socioeconomic_Status,Parent_Education_Level,Assignments_Completed
0,192,22,78,4,12.4,Public,Urban,Yes,Yes,High,Medium,1,17,Male,Low,Tertiary,2
1,207,14,88,4,2.7,Public,Rural,No,Yes,High,High,2,15,Male,High,NaN,1
2,182,29,87,2,9.6,Public,Rural,Yes,Yes,High,Medium,3,20,Female,High,Tertiary,2
3,210,29,99,2,2.6,Public,Urban,No,Yes,Medium,High,4,22,Female,Medium,Tertiary,1
4,199,12,98,3,8.8,Public,Urban,No,Yes,Medium,Medium,5,22,Female,Medium,Tertiary,1


In [3]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

### Preparation

    Remove the student_id column.
    Fill missing values with zeros.
    Do train/validation/test split with 60%/20%/20% distribution.
    Use the train_test_split function and set the random_state parameter to 1.
    Use DictVectorizer(sparse=True) to turn the dataframes into matrices.


In [4]:
df = df.fillna(0)
df.isna().sum()

,0
jamb_score,0
study_hours_per_week,0
attendance_rate,0
teacher_quality,0
distance_to_school,0
school_type,0
school_location,0
extra_tutorials,0
access_to_learning_materials,0
parent_involvement,0


In [5]:
del df['student_id']

In [6]:
from sklearn.model_selection import train_test_split
df_fulltrain, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_fulltrain, test_size=0.25, random_state=1)

In [7]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.jamb_score.values
y_val = df_val.jamb_score.values
y_test = df_test.jamb_score.values

del df_train['jamb_score']
del df_val['jamb_score']
del df_test['jamb_score']

In [8]:
from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer(sparse=True)
train_dict = df_train.to_dict(orient='records')
val_dict = df_val.to_dict(orient='records')

X_train = dv.fit_transform(train_dict)
X_val = dv.transform(val_dict)

# Q.1

In [9]:
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor(max_depth=1)
dt.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=1)

In [10]:
from sklearn.tree import export_text
print(export_text(dt, feature_names=dv.get_feature_names_out()))

|--- study_hours_per_week <= 18.50
|   |--- value: [155.24]
|--- study_hours_per_week >  18.50
|   |--- value: [188.59]



# Ans.  study_hours_per_week

# Q.2

#### Train a random forest model with these parameters:

    n_estimators=10
    random_state=1
    n_jobs=-1 (optional - to make training faster)

What's the RMSE of this model on validation?

In [11]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)
rf.fit(X_train, y_train)

RandomForestRegressor(n_estimators=10, n_jobs=-1, random_state=1)

In [12]:
from sklearn.metrics import mean_squared_error
y_pred = rf.predict(X_val)
rmse = mean_squared_error(y_val, y_pred, squared=False)
print(f'The rmse is: {rmse}')

The rmse is: 42.13724207871227


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


## Ans. 42.137

# Q.3

#### Now let's experiment with the n_estimators parameter

    Try different values of this parameter from 10 to 200 with step 10.
    Set random_state to 1.
    Evaluate the model on the validation dataset.

After which value of n_estimators does RMSE stop improving? Consider 3 decimal places for calculating the answer.

In [13]:
params = list(range(10, 201, 10))

In [14]:
import matplotlib.pyplot as plt
%matplotlib inline

In [15]:
n_estimators = []
for param in params:
    rf = RandomForestRegressor(n_estimators=param, random_state=1, n_jobs=-1)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    n_estimators.append({'n_estimator': param, 'rmse': rmse})
data = pd.DataFrame(n_estimators)
print(data)
print(f'The max rmse is: {data.rmse.max()}')

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'roo

    n_estimator       rmse
0            10  42.137242
1            20  41.461215
2            30  41.106171
3            40  40.917194
4            50  40.852279
5            60  40.784281
6            70  40.677098
7            80  40.539333
8            90  40.504346
9           100  40.516805
10          110  40.593353
11          120  40.624850
12          130  40.650841
13          140  40.594852
14          150  40.596715
15          160  40.603508
16          170  40.627546
17          180  40.641314
18          190  40.631355
19          200  40.601019
The max rmse is: 42.13724207871227


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


## Ans. 10

# Q.4

#### Let's select the best max_depth:

    Try different values of max_depth: [10, 15, 20, 25]
    For each of these values,
        try different values of n_estimators from 10 till 200 (with step 10)
        calculate the mean RMSE
    Fix the random seed: random_state=1

What's the best max_depth, using the mean RMSE?

In [16]:
depths = [10, 15, 20, 25]
mean_rmse = []
for depth in depths:
  n_estimators = []
  params = list(range(10, 201, 10))
  for param in params:
    rf = RandomForestRegressor(max_depth=depth, n_estimators=param, random_state=1)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    n_estimators.append({'depth': depth, 'n_estimator': param, 'rmse': rmse})
  data = pd.DataFrame(n_estimators)
  print(data)
  print(f'The mean rmse is: {data.rmse.mean()}')
  mean_rmse.append({'depth': depth, 'rmse': data.rmse.mean()})
print(pd.DataFrame(mean_rmse))


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'roo

    depth  n_estimator       rmse
0      10           10  41.258349
1      10           20  40.880528
2      10           30  40.624956
3      10           40  40.270069
4      10           50  40.317006
5      10           60  40.277035
6      10           70  40.285211
7      10           80  40.210305
8      10           90  40.173900
9      10          100  40.250490
10     10          110  40.286316
11     10          120  40.315324
12     10          130  40.328961
13     10          140  40.299757
14     10          150  40.314327
15     10          160  40.354350
16     10          170  40.360085
17     10          180  40.364196
18     10          190  40.353781
19     10          200  40.325012
The mean rmse is: 40.39249798892396


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'roo

    depth  n_estimator       rmse
0      15           10  42.003979
1      15           20  41.455699
2      15           30  41.167578
3      15           40  40.930593
4      15           50  40.782970
5      15           60  40.724170
6      15           70  40.689415
7      15           80  40.533749
8      15           90  40.496685
9      15          100  40.504812
10     15          110  40.530819
11     15          120  40.580236
12     15          130  40.557808
13     15          140  40.520364
14     15          150  40.529308
15     15          160  40.523299
16     15          170  40.533787
17     15          180  40.546689
18     15          190  40.550057
19     15          200  40.543618
The mean rmse is: 40.73528172486332


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'roo

    depth  n_estimator       rmse
0      20           10  42.024831
1      20           20  41.388831
2      20           30  41.116326
3      20           40  40.847429
4      20           50  40.808710
5      20           60  40.784463
6      20           70  40.653663
7      20           80  40.515399
8      20           90  40.493177
9      20          100  40.517604
10     20          110  40.572731
11     20          120  40.589681
12     20          130  40.589177
13     20          140  40.536444
14     20          150  40.537381
15     20          160  40.550345
16     20          170  40.564237
17     20          180  40.574707
18     20          190  40.581853
19     20          200  40.547698
The mean rmse is: 40.739734321829275


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'roo

    depth  n_estimator       rmse
0      25           10  42.160439
1      25           20  41.470970
2      25           30  41.125193
3      25           40  40.933939
4      25           50  40.866871
5      25           60  40.796605
6      25           70  40.687163
7      25           80  40.548492
8      25           90  40.512574
9      25          100  40.522283
10     25          110  40.591911
11     25          120  40.619898
12     25          130  40.646433
13     25          140  40.591027
14     25          150  40.593185
15     25          160  40.600208
16     25          170  40.624371
17     25          180  40.638564
18     25          190  40.628661
19     25          200  40.598527
The mean rmse is: 40.78786565962805
   depth       rmse
0     10  40.392498
1     15  40.735282
2     20  40.739734
3     25  40.787866


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


## Ans. 10

# Q.5

#### We can extract feature importance information from tree-based models.

At each step of the decision tree learning algorithm, it finds the best split. When doing it, we can calculate "gain" - the reduction in impurity before and after the split. This gain is quite useful in understanding what are the important features for tree-based models.

In Scikit-Learn, tree-based models contain this information in the feature_importances_ field.

For this homework question, we'll find the most important feature:

    Train the model with these parameters:
        n_estimators=10,
        max_depth=20,
        random_state=1,
        n_jobs=-1 (optional)
    Get the feature importance information from this model

What's the most important feature (among these 4)?

    study_hours_per_week
    attendance_rate
    distance_to_school
    teacher_quality

In [17]:
model = RandomForestRegressor(n_estimators=10, max_depth=20, random_state=1, n_jobs=-1)
model.fit(X_train, y_train)
model.feature_importances_
feature_names = dv.get_feature_names_out()
feature_importance = pd.DataFrame({'feature': feature_names, 'importance': model.feature_importances_})

In [18]:
feature_importance = feature_importance.sort_values(by='importance', ascending=False)
feature_importance

,feature,importance
27,study_hours_per_week,0.248354
4,attendance_rate,0.149729
5,distance_to_school,0.136486
28,teacher_quality,0.082682
2,age,0.069311
3,assignments_completed,0.031517
24,socioeconomic_status=High,0.025714
17,parent_involvement=High,0.022919
10,it_knowledge=High,0.017719
15,parent_education_level=Secondary,0.016957


## Ans.  study_hours_per_week

# Q.6

#### Now let's train an XGBoost model! For this question, we'll tune the eta parameter:

    Install XGBoost
    Create DMatrix for train and validation
    Create a watchlist
    Train a model with these parameters for 100 rounds:

In [19]:
xgb_params = {
    'eta': 0.3,
    'max_depth': 6,
    'min_child_weight': 1,
    'eval_metric': 'rmse',

    'objective': 'reg:squarederror',
    'nthread': 8,

    'seed': 1,
    'verbosity': 1,
}

In [20]:
import xgboost as xgb
features = list(dv.get_feature_names_out())
dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=features)
dval = xgb.DMatrix(X_val, label=y_val, feature_names=features)
watchlist = [(dtrain, 'train'), (dval, 'val')]
model = xgb.train(xgb_params, dtrain, num_boost_round=100, evals=watchlist)
y_pred = model.predict(dval)
rmse = mean_squared_error(y_val, y_pred, squared=False)
print(f'The rmse is: {rmse}')


[0]	train-rmse:42.69384	val-rmse:44.89114
[1]	train-rmse:39.83326	val-rmse:43.07010
[2]	train-rmse:37.94542	val-rmse:42.00332
[3]	train-rmse:36.56125	val-rmse:41.46452
[4]	train-rmse:35.44252	val-rmse:40.88896
[5]	train-rmse:34.57756	val-rmse:40.69096
[6]	train-rmse:33.84230	val-rmse:40.59315
[7]	train-rmse:33.25929	val-rmse:40.47993
[8]	train-rmse:32.79415	val-rmse:40.45326
[9]	train-rmse:32.16019	val-rmse:40.43929
[10]	train-rmse:31.63404	val-rmse:40.48319
[11]	train-rmse:31.17673	val-rmse:40.68201
[12]	train-rmse:30.87313	val-rmse:40.63522
[13]	train-rmse:30.30310	val-rmse:40.70983
[14]	train-rmse:30.00098	val-rmse:40.78133
[15]	train-rmse:29.41497	val-rmse:40.86107
[16]	train-rmse:29.25816	val-rmse:40.96580
[17]	train-rmse:28.59378	val-rmse:41.12190
[18]	train-rmse:28.27990	val-rmse:41.14360
[19]	train-rmse:27.94572	val-rmse:41.22835
[20]	train-rmse:27.49658	val-rmse:41.27921
[21]	train-rmse:27.25449	val-rmse:41.32427
[22]	train-rmse:27.06652	val-rmse:41.41887
[23]	train-rmse:26.78

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [21]:
%%capture output
model1 = xgb.train(xgb_params, dtrain, num_boost_round=100, evals=watchlist)

In [22]:
s = output.stdout
print(s[:100])

[0]	train-rmse:42.69384	val-rmse:44.89114
[1]	train-rmse:39.83326	val-rmse:43.07010
[2]	train-rmse:3


#### Now change eta from 0.3 to 0.1.

Which eta leads to the best RMSE score on the validation dataset?

In [23]:
print(s)

[0]	train-rmse:42.69384	val-rmse:44.89114
[1]	train-rmse:39.83326	val-rmse:43.07010
[2]	train-rmse:37.94542	val-rmse:42.00332
[3]	train-rmse:36.56125	val-rmse:41.46452
[4]	train-rmse:35.44252	val-rmse:40.88896
[5]	train-rmse:34.57756	val-rmse:40.69096
[6]	train-rmse:33.84230	val-rmse:40.59315
[7]	train-rmse:33.25929	val-rmse:40.47993
[8]	train-rmse:32.79415	val-rmse:40.45326
[9]	train-rmse:32.16019	val-rmse:40.43929
[10]	train-rmse:31.63404	val-rmse:40.48319
[11]	train-rmse:31.17673	val-rmse:40.68201
[12]	train-rmse:30.87313	val-rmse:40.63522
[13]	train-rmse:30.30310	val-rmse:40.70983
[14]	train-rmse:30.00098	val-rmse:40.78133
[15]	train-rmse:29.41497	val-rmse:40.86107
[16]	train-rmse:29.25816	val-rmse:40.96580
[17]	train-rmse:28.59378	val-rmse:41.12190
[18]	train-rmse:28.27990	val-rmse:41.14360
[19]	train-rmse:27.94572	val-rmse:41.22835
[20]	train-rmse:27.49658	val-rmse:41.27921
[21]	train-rmse:27.25449	val-rmse:41.32427
[22]	train-rmse:27.06652	val-rmse:41.41887
[23]	train-rmse:26.78

In [36]:
def parse_output(output):
  rmse_list = []
  res = output.split('\n')
  for item in res:
    elements = item.split('\t')
    if len(elements) == 3:
      train_rmse = elements[1].split(':')[1]
      val_rmse = elements[2].split(':')[1]
      rmse_list.append({'train_rmse': train_rmse, 'val_rmse': val_rmse})
  rmse_df = pd.DataFrame(rmse_list)
  print(rmse_df)
  print(f'The lowest val_rmse is: {rmse_df.val_rmse.min()}')
  print(f'This is the round for the lowest rmse: {rmse_df[rmse_df.val_rmse == rmse_df.val_rmse.min()]}')
  return pd.DataFrame(rmse_list)

In [41]:
rmse_df = parse_output(s)

   train_rmse  val_rmse
0    42.69384  44.89114
1    39.83326  43.07010
2    37.94542  42.00332
3    36.56125  41.46452
4    35.44252  40.88896
..        ...       ...
95   11.95568  43.37919
96   11.76819  43.39410
97   11.58218  43.40203
98   11.53812  43.40245
99   11.39140  43.41882

[100 rows x 2 columns]
The lowest val_rmse is: 40.43929
This is the round for the lowest rmse:   train_rmse  val_rmse
9   32.16019  40.43929


In [42]:
xgb_params2 = {
    'eta': 0.1,
    'max_depth': 6,
    'min_child_weight': 1,
    'eval_metric': 'rmse',

    'objective': 'reg:squarederror',
    'nthread': 8,

    'seed': 1,
    'verbosity': 1,
}
model2 = xgb.train(xgb_params2, dtrain, num_boost_round=100, evals=[(dval, 'val')])
y_pred = model2.predict(dval)
rmse2 = mean_squared_error(y_val, y_pred, squared=False)
print(f'The rmse for eta at 0.1 is: {rmse2}')


[0]	val-rmse:47.00533
[1]	val-rmse:45.92344
[2]	val-rmse:44.98366
[3]	val-rmse:44.25755
[4]	val-rmse:43.57339
[5]	val-rmse:43.11181
[6]	val-rmse:42.61054
[7]	val-rmse:42.18883
[8]	val-rmse:41.86754
[9]	val-rmse:41.64338
[10]	val-rmse:41.39235
[11]	val-rmse:41.14265
[12]	val-rmse:40.95201
[13]	val-rmse:40.81778
[14]	val-rmse:40.75008
[15]	val-rmse:40.61341
[16]	val-rmse:40.51800
[17]	val-rmse:40.41659
[18]	val-rmse:40.33546
[19]	val-rmse:40.25632
[20]	val-rmse:40.25010
[21]	val-rmse:40.19826
[22]	val-rmse:40.21101
[23]	val-rmse:40.14758
[24]	val-rmse:40.15079
[25]	val-rmse:40.12003
[26]	val-rmse:40.08693
[27]	val-rmse:40.08567
[28]	val-rmse:40.07263
[29]	val-rmse:40.14000
[30]	val-rmse:40.13806
[31]	val-rmse:40.11982
[32]	val-rmse:40.13489
[33]	val-rmse:40.16486
[34]	val-rmse:40.17118
[35]	val-rmse:40.16103
[36]	val-rmse:40.19895
[37]	val-rmse:40.19394
[38]	val-rmse:40.20019
[39]	val-rmse:40.18898
[40]	val-rmse:40.17753
[41]	val-rmse:40.19907
[42]	val-rmse:40.23107
[43]	val-rmse:40.2442

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [43]:
%%capture output2
model2 = xgb.train(xgb_params2, dtrain, num_boost_round=100, evals=watchlist)

In [44]:
s2 = output2.stdout
rmse_df2 = parse_output(s2)

   train_rmse  val_rmse
0    45.49999  47.00533
1    44.12948  45.92344
2    42.94858  44.98366
3    41.90896  44.25755
4    40.96728  43.57339
..        ...       ...
95   24.03404  40.99952
96   24.00230  41.00887
97   23.88940  41.05015
98   23.76232  41.06328
99   23.59704  41.05034

[100 rows x 2 columns]
The lowest val_rmse is: 40.07263
This is the round for the lowest rmse:    train_rmse  val_rmse
28   32.02459  40.07263


## Ans.  .1 has a lower rmse